In [ ]:
# from google.colab import drive
import sys

# drive.mount('/content/gdrive/')
# path = '/content/gdrive/MyDrive/Processamento Dados Conti/código/código/'
path='./'
sys.path.append(path)

## Cria, caso não exista, a pasta que armazenará os logs

In [ ]:
import os
import pandas as pd
import numpy as np
from preprocess.cleanup import filter_consumption, filter_odometer, filter_liter_supply, filter_km
from preprocess.cleanup import filter_o2, filter_nox_avg, filter_nox_max, filter_nox_min, filter_nox_std
from preprocess.cleanup import filter_nox_duplicates
from preprocess.cleanup import filter_desligado, filter_coordinate_in_brazil

if not os.path.exists(path+"dados limpos/"): os.makedirs(path+"dados limpos/")
nox_exists = os.path.exists(path+"dados/nox.csv")
abastecimento_exists = os.path.exists(path+"dados/abastecimentos.csv")

if nox_exists and abastecimento_exists:
    veiculos = pd.read_csv(path+'dados/informacoes_veiculos.csv', index_col=[0])
    abastecimento = pd.read_csv(path+'dados/abastecimentos.csv', index_col=[0])

## Código de limpeza
- Aplica todas as regras e vai filtrando o Dataframe
- Versões posteriores salvarão o que passar por cada regra específica
### Abastecimento

In [ ]:

import matplotlib.pyplot as plt
old_size = len(abastecimento)
abastecimento = filter_km(abastecimento, arg_return=0)
print(len(abastecimento))
abastecimento = filter_consumption(abastecimento, arg_return=0)
print(len(abastecimento))
# abastecimento = filter_liter_supply(abastecimento, veiculos, arg_return=0)
# print(len(abastecimento))
abastecimento = filter_odometer(abastecimento, arg_return=0)
print(len(abastecimento))
abastecimento.to_csv(path+'dados limpos/abastecimentos.csv')
new_size = len(abastecimento)

labels = 'Sem Problemas', 'Limpo'
sizes = [new_size, old_size-new_size]
fig, ax = plt.subplots()
ax.pie(sizes, labels=labels, autopct='%1.1f%%')

### NOx

In [ ]:

nox = pd.read_csv(path+'dados/nox.csv', index_col=[0])
old_size = len(nox)
nox = filter_o2(nox, arg_return=0)
print(len(nox))
nox = filter_nox_avg(nox, arg_return=0)
print(len(nox))
nox = filter_nox_max(nox, arg_return=0)
print(len(nox))
nox = filter_nox_min(nox, arg_return=0)
print(len(nox))
nox = filter_nox_std(nox, arg_return=0)
print(len(nox))
nox = filter_nox_duplicates(nox, arg_return=0)
print(len(nox))
# nox = filter_coordinate_duplicates(nox, arg_return=0)
print(len(nox))
nox = filter_desligado(nox, arg_return=0)
print(len(nox))
nox = filter_coordinate_in_brazil(nox, arg_return=0)
print(len(nox))
nox.to_csv(path+'dados limpos/nox.csv')
new_size = len(nox)

import matplotlib.pyplot as plt
labels = 'Sem Problemas', 'Limpo'
sizes = [new_size, old_size-new_size]
fig, ax = plt.subplots()
ax.pie(sizes, labels=labels, autopct='%1.1f%%')

## Código de limpeza complementar
- Célula que salva dados que se encaixam nas regras
### Abastecimento

Primeiramente precisamos adicionar uma cópia sem mudanças ao procedimento

In [ ]:
def full_csv(select = 'abastecimento'):
    nox_exists = os.path.exists(path+"dados/nox.csv")
    abastecimento_exists = os.path.exists(path+"dados/abastecimentos.csv")
    if select == 'nox' and nox_exists:
        return pd.read_csv(path+'dados/nox.csv')
    elif select == 'abastecimento' and abastecimento_exists:
        return pd.read_csv(path+'dados/abastecimentos.csv')

Agora, aplicamos as regras de limpeza à locais diferentes

In [ ]:

if not os.path.exists(path+'dados limpos/invalidos/'): os.makedirs(path+'dados limpos/invalidos/')
filter_km(full_csv('abastecimento'), arg_return=1).to_csv(path+"dados limpos/invalidos/invalid_km.csv", index=False)
filter_odometer(full_csv('abastecimento'), arg_return=1).to_csv(path+"dados limpos/invalidos/inverted_odometer.csv", index=False)
filter_consumption(full_csv('abastecimento'), arg_return=1).to_csv(path+"dados limpos/invalidos/invalid_consumption.csv", index=False)
# filter_liter_supply(full_csv('abastecimento'), veiculos, arg_return=1).to_csv(path+"dados limpos/invalidos/invalid_tank.csv", index=False)

E o mesmo se aplica ao NOx

### NOx

In [ ]:
filter_o2(full_csv('nox'), arg_return=1).to_csv(path+"dados limpos/invalidos/invalid_o2.csv", index=False)
filter_nox_avg(full_csv('nox'), arg_return=1).to_csv(path+"dados limpos/invalidos/invalid_nox.csv", index=False)
filter_desligado(full_csv('nox'), arg_return=1).to_csv(path+"dados limpos/invalidos/vehicle_off.csv", index=False)
filter_nox_max(full_csv('nox'), arg_return=1).to_csv(path+"dados limpos/invalidos/invalid_nox_max.csv", index=False)
filter_nox_min(full_csv('nox'), arg_return=1).to_csv(path+"dados limpos/invalidos/invalid_nox_min.csv", index=False)
filter_nox_std(full_csv('nox'), arg_return=1).to_csv(path+"dados limpos/invalidos/invalid_nox_std.csv", index=False)
filter_nox_duplicates(full_csv('nox'), arg_return=1).to_csv(path+"dados limpos/invalidos/nox_duplicates.csv", index=False)
filter_coordinate_in_brazil(full_csv('nox'), arg_return=1).to_csv(path+"dados limpos/invalidos/outside_brazil.csv", index=False)
# filter_coordinate_duplicates(full_csv('nox'), arg_return=1).to_csv(path+"dados limpos/invalidos/coordinate_duplicate.csv", index=False)